<a href="https://colab.research.google.com/github/camilorey/material_clases/blob/main/tablasDeContingencia_datosBogota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as Pandas
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Debido a la gran cantidad de datos y su estructura, este dataset se presta mucho para almacenarlo en una Base de Datos. Este dataset en últimas tiene solo una variable numérica (la edad) y tanto la fecha como la hora son tipo TIMESTAMP. Mientras que los datasets pesaban en total 59MB, gracias a la factorización hecha en la DB pasaron a pesar apenas 3MB en total. 

Asi, tomé la decisión de tercerizar esos datos a una Base de Datos para comprimirla. La base de datos está subida en un SaaS de PostgreSQL llamado ElephantSQL que les permite hostear hasta 20MB de información. El diagrama de la DB lo pueden ver [aquí](https://https://dbdiagram.io/d/5e7cfa144495b02c3b88d373). 

In [ ]:
#conexión a la Base de Datos
#parametros de conexion a la DB
parametrosDict = {
    "host":"drona.db.elephantsql.com",
    "database":"autkzlzm",
    "user":"autkzlzm",
    "password":"i9bSQBZCdBn79D74lWGdGVROK6eYuBQn"
}
#esto retorna queries como dataframes
def queryComoDataFrame(sqlQuery):
  DBConnection = None #comenzamos creando un objeto conexión para conectarnos a la DB con los parámetros dados
  resultDataFrame = None #comenzamos con un objeto dataframe vacío.
  try:
    # conectar usando el método connect de pyscopg2
    print('Connecting to the PostgreSQL database...')
    DBConnection = psycopg2.connect(**parametrosDict)
    #usar el Pandas DataFrame para recibir el comando
    resultDataFrame = Pandas.read_sql_query(sqlQuery, DBConnection)
    DBConnection.close()
    return resultDataFrame
  except (Exception, psycopg2.DatabaseError) as error:
    print('Error en el Query:',error)
    return None
  #si algo no ha sido cubierto en los casos anteriores, cerrar la conexión a la base de datos
  finally:
    if DBConnection is not None:
      DBConnection.close()
    print('en caso finally: query ejecutado, resultados en un data frame.')

Para construir una tabla de contingencia sobre dos variables categóricas debemos contar incidencias de las categorías de ambas. Queremos ver independencia entre estas variables categóricas. 

**Para este dataset en particular, estamos viendo la independencia entre localidades y años, entre años de incidentes criminales y si la incidencia criminal es independiente del año. (la pregunta es filosófica). **

En este caso, usé una función SQL para contar el número de incidentes en una localidad por año. Pillen el código para contar el número de incidentes en un año y en una localidad:

```
CREATE OR REPLACE FUNCTION numIncidentesEnLocalidadAno(codLocalidad VARCHAR, ano INTEGER) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
BEGIN
 RETURN count(incidentes.incidente_id)
 FROM incidentes
 WHERE incidentes.codigo_localidad = codLocalidad AND EXTRACT(YEAR FROM incidentes.fecha) = ano;
END;
$$;
```
Esto se puede exportar a Python perfectamente. Sin embargo, prefería mantenerlo en el *Back-End*. Usando esta función es fácil generar una vista SQL que es donde voy a construir la tabla de contingencia 
```
CREATE VIEW incidentesAnuales AS
SELECT localidades.codigo_localidad,
       localidades.nombre_localidad,
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2010) as "2010",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2012) as "2012",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2013) as "2013",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2014) as "2014",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2015) as "2015",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2018) as "2018",
       numIncidentesEnLocalidadAno(localidades.codigo_localidad, 2019) as "2019"
from localidades
group by localidades.codigo_localidad
ORDER BY localidades.codigo_localidad;
```
De modo tal que puedo extraer esa tabla de contingencia de la Base de Datos ya estructurada y desde Python me dedico a hacer cálculos.

In [ ]:
#vamos a mostrar por año el número de incidentes
queryIncidentesAnuales = queryComoDataFrame("SELECT  * from incidentesAnuales")
queryIncidentesAnuales 

Connecting to the PostgreSQL database...
en caso finally: query ejecutado, resultados en un data frame.


,codigo_localidad,nombre_localidad,2010,2012,2013,2014,2015,2018,2019
0,01,USAQUEN,55,61,36,69,326,3028,4494
1,02,CHAPINERO,23,22,14,32,128,46,19
2,03,SANTA FE,34,21,25,26,88,53,7
3,04,SAN CRISTOBAL,56,69,104,86,142,139,24
4,05,USME,74,67,74,80,132,114,46
5,06,TUNJUELITO,32,33,36,32,61,50,18
6,07,BOSA,97,105,86,137,228,200,211
7,08,KENNEDY,151,145,116,136,292,245,84
8,09,FONTIBON,38,46,37,36,107,63,86
9,10,ENGATIVA,69,54,74,84,320,141,42


Ahora, podemos limpiar este DataFrame para quitar campos que no nos interesan o que podrían arruinar el cálculo. A saber no hay incidentes registrados en la Localidad de Sumapaz y de nada nos sirve el código de localidad. 

In [ ]:
#vamos a quitar el registro de la localidad de Sumapaz que no ofrece nada de información
queryIncidentesAnuales = queryIncidentesAnuales.loc[queryIncidentesAnuales['nombre_localidad']!='SUMAPAZ']


Tablas de contingencia se pueden hacer en Excel, R, Stata, etc. Lo bonito de hacerlas en Python es que podemos hacer tablas *asociativas* usando los nombres de las categorías de la variable como hacemos aquí con la función set_index(). 

Esto nos va a permitir hacer las sumas que debemos hacer más fácilmente. 

In [ ]:
queryIncidentesAnuales = queryIncidentesAnuales.drop(columns=['codigo_localidad'])
queryIncidentesAnuales = queryIncidentesAnuales.set_index('nombre_localidad')
queryIncidentesAnuales

,2010,2012,2013,2014,2015,2018,2019
nombre_localidad,,,,,,,
USAQUEN,55,61,36,69,326,3028,4494
CHAPINERO,23,22,14,32,128,46,19
SANTA FE,34,21,25,26,88,53,7
SAN CRISTOBAL,56,69,104,86,142,139,24
USME,74,67,74,80,132,114,46
TUNJUELITO,32,33,36,32,61,50,18
BOSA,97,105,86,137,228,200,211
KENNEDY,151,145,116,136,292,245,84
FONTIBON,38,46,37,36,107,63,86


Para hacer una prueba de independencia en tablas de contingencia debemos calcular la suma por fila. Comenzamos con la suma por columnas. Estas nos dan la base para calcular probabilidades, usando el viejo esquema de 

>>>$p = \frac{\text{eventos favorables}}{\text{eventos totales}}$

In [ ]:
#vamos a generar un diccionario que contenga la suma por filas
sumaLocalidades = dict(queryIncidentesAnuales.sum(axis = 1, skipna = True))
#vamos a generar un diccionario que contenga la suma por columnas
sumaAnos = dict(queryIncidentesAnuales.sum(axis=0,skipna=True))

Ahora debemos calcular el gran total por fila y por columna

In [ ]:
totalLocalidades = 0
totalAnos = 0
for loc in sumaLocalidades.keys():
  totalLocalidades += sumaFilas[loc]
for ano in sumaAnos.keys():
  totalAnos += sumaAnos[ano]
print(totalLocalidades, totalAnos)
granTotal = totalLocalidades

17647 17647


Ahora podemos construir la *tabla de probabilidades* con esta información. Cada posición de la tabla de contingencia está dada por
>> $t_{ij} = \frac{S_iS_j}{T}$ 

donde $i$ denota la i-ésima fila y $j$ la j-ésima columna. Este cálculo deriva de la regla de probabilidad condicional. 

In [ ]:
localidades = list(sumaLocalidades.keys())
anos = list(sumaAnos.keys())
tablaContingencia = Pandas.DataFrame(columns = ['nombre_localidad']+anos)
for loc in localidades:
  filaDict = {}
  filaDict['nombre_localidad'] = loc 
  for ano in anos:
    contingencia = sumaLocalidades[loc]*sumaAnos[ano]/granTotal
    filaDict[ano] = contingencia
  tablaContingencia = tablaContingencia.append(filaDict,ignore_index=True)
tablaContingencia = tablaContingencia.set_index('nombre_localidad')
tablaContingencia

,2010,2012,2013,2014,2015,2018,2019
nombre_localidad,,,,,,,
USAQUEN,478.735366,503.426588,484.222304,542.292401,1379.964980,2209.407151,2470.951210
CHAPINERO,16.849776,17.718819,17.042897,19.086757,48.569842,77.763246,86.968663
SANTA FE,15.069870,15.847113,15.242591,17.070550,43.439225,69.548818,77.781833
SAN CRISTOBAL,36.784723,38.681929,37.206324,41.668272,106.032753,169.764833,189.861166
USME,34.826826,36.623052,35.225987,39.450445,100.389075,160.728962,179.755653
TUNJUELITO,15.544512,16.346234,15.722672,17.608205,44.807389,71.739332,80.231654
BOSA,63.127330,66.383181,63.850853,71.508132,181.965887,291.338358,325.826259
KENNEDY,69.357001,72.934153,70.151924,78.564855,199.923046,320.088854,357.980167
FONTIBON,24.503372,25.767156,24.784213,27.756446,70.631495,113.085284,126.472035


Ahora, el estadístico matricial de prueba es la suma de la diferencia entre la tabla original y la contigencia (una suerte de norma matricial). 

La fórmula para la casilla de la matriz que necesitamos es:
>>$r_{ij} = \frac{(A_{ij}-T_{ij})^2}{T_{ij}}$

Noten que esto es una suerte de error porcentual (elevando al cuadrado)

In [ ]:
tablaDiferencia = (queryIncidentesAnuales - tablaContingencia)**2/tablaContingencia
tablaDiferencia

,2010,2012,2013,2014,2015,2018,2019
nombre_localidad,,,,,,,
USAQUEN,375.054097,388.817934,414.898761,413.071797,804.978528,303.291429,1656.336390
CHAPINERO,2.244852,1.034409,0.543289,8.736521,129.898508,12.974044,53.119584
SANTA FE,23.779224,1.675526,6.246119,4.670914,45.711283,3.937715,64.411800
SAN CRISTOBAL,10.037506,23.762658,119.909592,47.165433,12.200408,5.575212,144.894962
USME,44.061941,25.196124,42.679401,41.679287,9.953778,13.585578,99.527188
TUNJUELITO,17.419852,16.967082,26.151408,11.762911,5.851728,6.587719,48.269960
BOSA,18.175293,22.464405,7.683292,59.981777,11.645807,28.635761,40.466566
KENNEDY,96.105355,71.207878,29.964197,41.988187,42.407144,17.614909,209.690756
FONTIBON,7.434037,15.887201,6.020989,2.448303,18.726322,22.182689,12.951366


In [ ]:
from scipy import stats
gradosDeLibertad = (len(tablaDiferencia)-1)*(len(tablaDiferencia.columns)-1)
valorObtenido = tablaDiferencia.values.sum()
print("grados de libertad:", len(tablaDiferencia)-1,"X",len(tablaDiferencia.columns)-1,"=",gradosDeLibertad)
print("Estadístico de prueba:",valorObtenido)
chi2_estadistico, p_values, dof, ex = stats.chi2_contingency(queryIncidentesAnuales.values)
print("===Chi2 Stat===")
print("Estadístico Chi2:",chi2_estadistico)
print("grados de libertad:",dof)
print("p-value",p_val)

grados de libertad: 19 X 6 = 114
Estadístico de prueba: 9162.950394857253
===Chi2 Stat===
Estadístico Chi2: 9162.950394857253
grados de libertad: 114
p-value 0.0


Como el p-value es 0 es muy poco probable que las variables sean independientes. 